In [83]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [84]:
training_data = pd.read_csv("train.csv")
# testing_data = pd.read_csv("test.csv")

## Treinamento

### Tratamento dos dados

In [85]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [86]:
# removendo as colunas que não são úteis
training_data.drop(["SSN", "Name", "Type_of_Loan"], axis=1, inplace=True)

#### Tratando as variáveis categóricas

In [87]:
categorical_cols = [j for j in training_data.columns if training_data[j].dtype == "object"]

In [88]:
for j in categorical_cols:
    print(f"Valores únicos para '{j}'")
    print(training_data[j].unique())
    print("==================================================")

Valores únicos para 'ID'
['0x1602' '0x1603' '0x1604' ... '0x25feb' '0x25fec' '0x25fed']
Valores únicos para 'Customer_ID'
['CUS_0xd40' 'CUS_0x21b1' 'CUS_0x2dbc' ... 'CUS_0xaf61' 'CUS_0x8600'
 'CUS_0x942c']
Valores únicos para 'Month'
['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August']
Valores únicos para 'Age'
['23' '-500' '28_' ... '4808_' '2263' '1342']
Valores únicos para 'Occupation'
['Scientist' '_______' 'Teacher' 'Engineer' 'Entrepreneur' 'Developer'
 'Lawyer' 'Media_Manager' 'Doctor' 'Journalist' 'Manager' 'Accountant'
 'Musician' 'Mechanic' 'Writer' 'Architect']
Valores únicos para 'Annual_Income'
['19114.12' '34847.84' '34847.84_' ... '20002.88' '39628.99' '39628.99_']
Valores únicos para 'Num_of_Loan'
['4' '1' '3' '967' '-100' '0' '0_' '2' '3_' '2_' '7' '5' '5_' '6' '8' '8_'
 '9' '9_' '4_' '7_' '1_' '1464' '6_' '622' '352' '472' '1017' '945' '146'
 '563' '341' '444' '720' '1485' '49' '737' '1106' '466' '728' '313' '843'
 '597_' '617' '119' '663' '640' '92_' '

In [89]:
for j in categorical_cols:
    # muitas dados possuem underscore entre seus valores, basta removê-los
    training_data[j] = training_data[j].str.strip("_")
    
    # substituindo dados faltantes por NaN
    training_data[j] = training_data[j].replace({"": np.nan})

    # algumas linhas possuem esses valores bizarros, basta substituir por NaN
    training_data[j] = training_data[j].replace({"!@9#%8": np.nan, "#F%$D@*&8": np.nan})

    try:
        # algumas colunas "categóricas" são de dados númericos, mas possuem o tipo 'object'
        training_data[j] = training_data[j].astype("float64")
    except:
        training_data[j] = training_data[j]

In [90]:
# o ano do histórico de crédito do cliente também é uma variável categórica baseada em ano e mês
# podemos converter essa coluna para uma coluna numérica que representa apenas a quantidade de anos do histórico

credit_history_age = []
for i in training_data["Credit_History_Age"]:
    credit_history_age.append(str(i).split()[0])
    
training_data["Credit_History_Age"] = credit_history_age
training_data["Credit_History_Age"] = training_data["Credit_History_Age"].replace({"nan" : np.nan})
training_data["Credit_History_Age"] = training_data["Credit_History_Age"].astype("float64")

In [91]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Age                       100000 non-null  float64
 4   Occupation                92938 non-null   object 
 5   Annual_Income             100000 non-null  float64
 6   Monthly_Inhand_Salary     84998 non-null   float64
 7   Num_Bank_Accounts         100000 non-null  int64  
 8   Num_Credit_Card           100000 non-null  int64  
 9   Interest_Rate             100000 non-null  int64  
 10  Num_of_Loan               100000 non-null  float64
 11  Delay_from_due_date       100000 non-null  int64  
 12  Num_of_Delayed_Payment    92998 non-null   float64
 13  Changed_Credit_Limit      97909 non-null   fl

#### Tratando as variáveis numéricas

In [92]:
numerical_cols = [col for col in training_data.columns if (training_data[col].dtype == "int64")
                                                        | (training_data[col].dtype == "float64")]

In [93]:
def outlier_treatment(column):
    Q1, Q3 = np.percentile(column, [25, 75])
    IQR = Q3 - Q1
    lower_range = Q1 - (1.5 * IQR)
    upper_range = Q3 + (1.5 * IQR)

    return lower_range, upper_range

In [94]:
# existem idades incorretas (negativas), bem como outros outliers
# utilizo o método IQR para substituir esses valores por NaN

for j in numerical_cols:
    lowerbound, upperbound = outlier_treatment(training_data[j])

    training_data.loc[training_data[j] < lowerbound, j] = np.nan
    training_data.loc[training_data[j] > upperbound, j] = np.nan

#### Tratando os dados faltantes

In [95]:
missing = training_data.isnull().sum()
missing[missing > 0]

Age                          2781
Occupation                   7062
Annual_Income                2783
Monthly_Inhand_Salary       15002
Num_Bank_Accounts            1315
Num_Credit_Card              2271
Interest_Rate                2034
Num_of_Loan                  4348
Delay_from_due_date          4002
Num_of_Delayed_Payment       7002
Changed_Credit_Limit         2091
Num_Credit_Inquiries         1965
Credit_Mix                  20195
Outstanding_Debt             5272
Credit_Utilization_Ratio        4
Credit_History_Age           9030
Total_EMI_per_month          6795
Amount_invested_monthly      4479
Payment_Behaviour            7600
Monthly_Balance              2868
dtype: int64

In [96]:
missing_cols = list(missing.index)

##### Tratando os dados númericos faltantes

In [99]:
# intersecção das colunas de variáveis númericas com as colunas com dados faltantes
numerical_cols_missing = list(set(numerical_cols).intersection(set(missing_cols)))

numerical_cols_missing.append("Customer_ID")

In [100]:
numerical_cols_missing.pop()

'Customer_ID'